# HW2 Path Planning and Collision Checking
Ting-Wei Hsu (twhsu3)

In [95]:
# Rigid body dynamics (pinocchio)
import pinocchio as pin

# Collision checking
import coal

# Visualization (meshcat)
from pinocchio.visualize import MeshcatVisualizer
import meshcat_shapes

# Robot models (robot_descriptions)
from robot_descriptions.loaders.pinocchio import load_robot_description

# Math ("expm" is the matrix exponential function)
import numpy as np
from scipy.linalg import expm, logm
from qpsolvers import solve_qp

# Timing
import time

# Plots
import matplotlib.pyplot as plt

# Interaction
import ipywidgets as widgets
from IPython.display import display
from tqdm.notebook import tqdm

# Other utilities
import itertools

import random

# Suppress the display of very small numbers
np.set_printoptions(suppress=True)

## Universal Robots UR10
The Universal Robots UR10 is a 6-joint collaborative robot arm.

Load a robot model.

In [65]:
# Load model
robot = load_robot_description(
    'ur10_description',           # name of robot model
    root_joint=None,                # fixed base
)

Add all possible collision pairs. Since the robot model has no obstacles yet, these pairs are only associated with self-collision.

In [66]:
robot.collision_model.addAllCollisionPairs()

Show all collision pairs.

In [67]:
for i, pair in enumerate(robot.collision_model.collisionPairs):
    print(f'{i:2d} : {pair.first:2d} x {pair.second:2d} ({robot.collision_model.geometryObjects[pair.first].name} x {robot.collision_model.geometryObjects[pair.second].name})')

 0 :  0 x  1 (base_link_0 x shoulder_link_0)
 1 :  0 x  2 (base_link_0 x upper_arm_link_0)
 2 :  0 x  3 (base_link_0 x forearm_link_0)
 3 :  0 x  4 (base_link_0 x wrist_1_link_0)
 4 :  0 x  5 (base_link_0 x wrist_2_link_0)
 5 :  0 x  6 (base_link_0 x wrist_3_link_0)
 6 :  0 x  7 (base_link_0 x ee_link_0)
 7 :  1 x  2 (shoulder_link_0 x upper_arm_link_0)
 8 :  1 x  3 (shoulder_link_0 x forearm_link_0)
 9 :  1 x  4 (shoulder_link_0 x wrist_1_link_0)
10 :  1 x  5 (shoulder_link_0 x wrist_2_link_0)
11 :  1 x  6 (shoulder_link_0 x wrist_3_link_0)
12 :  1 x  7 (shoulder_link_0 x ee_link_0)
13 :  2 x  3 (upper_arm_link_0 x forearm_link_0)
14 :  2 x  4 (upper_arm_link_0 x wrist_1_link_0)
15 :  2 x  5 (upper_arm_link_0 x wrist_2_link_0)
16 :  2 x  6 (upper_arm_link_0 x wrist_3_link_0)
17 :  2 x  7 (upper_arm_link_0 x ee_link_0)
18 :  3 x  4 (forearm_link_0 x wrist_1_link_0)
19 :  3 x  5 (forearm_link_0 x wrist_2_link_0)
20 :  3 x  6 (forearm_link_0 x wrist_3_link_0)
21 :  3 x  7 (forearm_link_0

Update robot data.

In [68]:
robot.data = robot.model.createData()
robot.collision_data = robot.collision_model.createData()
robot.visual_data = robot.visual_model.createData()

Put the robot at its neutral configuration and check for collisions.

In [69]:
# Get the neutral configuration
q = pin.neutral(robot.model)

# Update geometry placements at this configuration
pin.updateGeometryPlacements(
    robot.model,
    robot.data,
    robot.collision_model,
    robot.collision_data,
    q,
)

# Find all collisions
in_collision = pin.computeCollisions(robot.collision_model, robot.collision_data, False)

# Show all pairs in collision
for i, pair in enumerate(robot.collision_model.collisionPairs):
    s = f'{i:2d} : {pair.first:2d} x {pair.second:2d} ({robot.collision_model.geometryObjects[pair.first].name} x {robot.collision_model.geometryObjects[pair.second].name})'
    if robot.collision_data.collisionResults[i].isCollision():
        print(f'{s} : COLLISION')
    else:
        print(s)

 0 :  0 x  1 (base_link_0 x shoulder_link_0)
 1 :  0 x  2 (base_link_0 x upper_arm_link_0)
 2 :  0 x  3 (base_link_0 x forearm_link_0)
 3 :  0 x  4 (base_link_0 x wrist_1_link_0)
 4 :  0 x  5 (base_link_0 x wrist_2_link_0)
 5 :  0 x  6 (base_link_0 x wrist_3_link_0)
 6 :  0 x  7 (base_link_0 x ee_link_0)
 7 :  1 x  2 (shoulder_link_0 x upper_arm_link_0) : COLLISION
 8 :  1 x  3 (shoulder_link_0 x forearm_link_0)
 9 :  1 x  4 (shoulder_link_0 x wrist_1_link_0)
10 :  1 x  5 (shoulder_link_0 x wrist_2_link_0)
11 :  1 x  6 (shoulder_link_0 x wrist_3_link_0)
12 :  1 x  7 (shoulder_link_0 x ee_link_0)
13 :  2 x  3 (upper_arm_link_0 x forearm_link_0) : COLLISION
14 :  2 x  4 (upper_arm_link_0 x wrist_1_link_0)
15 :  2 x  5 (upper_arm_link_0 x wrist_2_link_0)
16 :  2 x  6 (upper_arm_link_0 x wrist_3_link_0)
17 :  2 x  7 (upper_arm_link_0 x ee_link_0)
18 :  3 x  4 (forearm_link_0 x wrist_1_link_0) : COLLISION
19 :  3 x  5 (forearm_link_0 x wrist_2_link_0)
20 :  3 x  6 (forearm_link_0 x wrist_3_

Create a hard-coded list of all pairs that are in collision at the neutral configuration (this could be automated). Then, remove these pairs from the list to check.

In [70]:
# Create a list of useless collision pairs (i.e., those pairs that are always
# in collision even at the neutral configuration)
useless_pairs = [7, 13, 18, 22, 25]

# Remove useless collision pairs - it is important that you remove these from
# last to first (i.e., from high to low index) because higher indices will change
# as you remove lower ones
for i in sorted(useless_pairs, reverse=True):
    robot.collision_model.removeCollisionPair(robot.collision_model.collisionPairs[i])

# Update robot data
robot.data = robot.model.createData()
robot.collision_data = robot.collision_model.createData()
robot.visual_data = robot.visual_model.createData()

Check for collisions again at the neutral configuration to make sure we did this correctly.

In [71]:
# Get the neutral configuration
q = pin.neutral(robot.model)

# Update geometry placements at this configuration
pin.updateGeometryPlacements(
    robot.model,
    robot.data,
    robot.collision_model,
    robot.collision_data,
    q,
)

# Find all collisions
in_collision = pin.computeCollisions(robot.collision_model, robot.collision_data, False)

# Show all pairs in collision
for i, pair in enumerate(robot.collision_model.collisionPairs):
    s = f'{i:2d} : {pair.first:2d} x {pair.second:2d} ({robot.collision_model.geometryObjects[pair.first].name} x {robot.collision_model.geometryObjects[pair.second].name})'
    if robot.collision_data.collisionResults[i].isCollision():
        print(f'{s} : COLLISION')
    else:
        print(s)

 0 :  0 x  1 (base_link_0 x shoulder_link_0)
 1 :  0 x  2 (base_link_0 x upper_arm_link_0)
 2 :  0 x  3 (base_link_0 x forearm_link_0)
 3 :  0 x  4 (base_link_0 x wrist_1_link_0)
 4 :  0 x  5 (base_link_0 x wrist_2_link_0)
 5 :  0 x  6 (base_link_0 x wrist_3_link_0)
 6 :  0 x  7 (base_link_0 x ee_link_0)
 7 :  1 x  3 (shoulder_link_0 x forearm_link_0)
 8 :  1 x  4 (shoulder_link_0 x wrist_1_link_0)
 9 :  1 x  5 (shoulder_link_0 x wrist_2_link_0)
10 :  1 x  6 (shoulder_link_0 x wrist_3_link_0)
11 :  1 x  7 (shoulder_link_0 x ee_link_0)
12 :  2 x  4 (upper_arm_link_0 x wrist_1_link_0)
13 :  2 x  5 (upper_arm_link_0 x wrist_2_link_0)
14 :  2 x  6 (upper_arm_link_0 x wrist_3_link_0)
15 :  2 x  7 (upper_arm_link_0 x ee_link_0)
16 :  3 x  5 (forearm_link_0 x wrist_2_link_0)
17 :  3 x  6 (forearm_link_0 x wrist_3_link_0)
18 :  3 x  7 (forearm_link_0 x ee_link_0)
19 :  4 x  6 (wrist_1_link_0 x wrist_3_link_0)
20 :  4 x  7 (wrist_1_link_0 x ee_link_0)
21 :  5 x  7 (wrist_2_link_0 x ee_link_0)


## Add obstacles and enable checking for collision with them

Record how many collision geometries are associated only with the robot.

In [72]:
n_robot_geoms = len(robot.collision_model.geometryObjects)
print(n_robot_geoms)

8


Define a function that allows us to specify a pose by $x, y, z$ coordinates and by body-fixed yaw ($\psi$), pitch ($\theta$), and roll ($\phi$) angles.

In [73]:
def XYZYPRtoSE3(xyzypr):
    R = pin.utils.rotate('z', xyzypr[3]) @ pin.utils.rotate('y', xyzypr[4]) @ pin.utils.rotate('x', xyzypr[5])
    p = np.array(xyzypr[:3])
    return pin.SE3(R, p)

Add a number of "capsules" as obstacles to the environment. If you want more complicated shapes, it is also possible to load meshes from files.

In [74]:
# Define the pose of each capsule
xyz_ypr = [
    [ 0.4,  0.0, 0.5, 0., 0., np.pi / 2],
    [-0.4, -0.0, 1.0, 0., 0., np.pi / 2],
    [-0.5,  0.0, 0.2, 0., 0., np.pi / 2],
]

# Define the radius and length of each capsule
rad_len = [
    [0.1, 0.4],
    [0.1, 0.4],
    [0.1, 0.4],
]

# Define the color of each capsule (all the same)
rgba = np.array([1.0, 0.2, 0.2, 1.0])

# Add each capsule as an obstacle that is fixed in space
for i, (xyz_ypr_i, rad_len_i) in enumerate(zip(xyz_ypr, rad_len)):
    # Define obstacle
    obs = pin.GeometryObject.CreateCapsule(rad_len_i[0], rad_len_i[1])
    obs.meshColor = rgba
    obs.name = f'obs{i}'
    obs.parentJoint = 0
    obs.placement = XYZYPRtoSE3(xyz_ypr_i)

    # Add obstacle to both the visual model and the collision model
    robot.collision_model.addGeometryObject(obs)
    robot.visual_model.addGeometryObject(obs)

Add collision pairs for all obstacles.

In [75]:
# Iterate over all robot geometries
for i in range(n_robot_geoms):
    # Iterate over all obstacle geometries
    for j in range(n_robot_geoms, len(robot.collision_model.geometryObjects)):
        robot.collision_model.addCollisionPair(pin.CollisionPair(i, j))

# Update robot data
robot.data = robot.model.createData()
robot.collision_data = robot.collision_model.createData()
robot.visual_data = robot.visual_model.createData()

Show all collision pairs again.

In [76]:
for i, pair in enumerate(robot.collision_model.collisionPairs):
    print(f'{i:2d} : {pair.first:2d} x {pair.second:2d} ({robot.collision_model.geometryObjects[pair.first].name} x {robot.collision_model.geometryObjects[pair.second].name})')

 0 :  0 x  1 (base_link_0 x shoulder_link_0)
 1 :  0 x  2 (base_link_0 x upper_arm_link_0)
 2 :  0 x  3 (base_link_0 x forearm_link_0)
 3 :  0 x  4 (base_link_0 x wrist_1_link_0)
 4 :  0 x  5 (base_link_0 x wrist_2_link_0)
 5 :  0 x  6 (base_link_0 x wrist_3_link_0)
 6 :  0 x  7 (base_link_0 x ee_link_0)
 7 :  1 x  3 (shoulder_link_0 x forearm_link_0)
 8 :  1 x  4 (shoulder_link_0 x wrist_1_link_0)
 9 :  1 x  5 (shoulder_link_0 x wrist_2_link_0)
10 :  1 x  6 (shoulder_link_0 x wrist_3_link_0)
11 :  1 x  7 (shoulder_link_0 x ee_link_0)
12 :  2 x  4 (upper_arm_link_0 x wrist_1_link_0)
13 :  2 x  5 (upper_arm_link_0 x wrist_2_link_0)
14 :  2 x  6 (upper_arm_link_0 x wrist_3_link_0)
15 :  2 x  7 (upper_arm_link_0 x ee_link_0)
16 :  3 x  5 (forearm_link_0 x wrist_2_link_0)
17 :  3 x  6 (forearm_link_0 x wrist_3_link_0)
18 :  3 x  7 (forearm_link_0 x ee_link_0)
19 :  4 x  6 (wrist_1_link_0 x wrist_3_link_0)
20 :  4 x  7 (wrist_1_link_0 x ee_link_0)
21 :  5 x  7 (wrist_2_link_0 x ee_link_0)
2

## Convert all collision geometries to convex meshes

This isn't strictly necessary. However, I have found that (1) it makes collision checking with coal much faster, and (2) it is useful for the purpose of example when we want to implement our own QP-based collision checker.

Define function to convert each primitive geometry to a convex mesh by sampling its surface.

In [77]:
def primitive_to_convex(shape, n=32):
    """
    Convert a coal primitive shape to a ConvexBase by sampling
    its surface, building a BVHModel from the points, and computing
    the convex hull.
    """
    if isinstance(shape, coal.Box):
        # half-extents
        x, y, z = shape.halfSide
        verts = np.array([
            [ x,  y,  z], [ x,  y, -z], [ x, -y,  z], [ x, -y, -z],
            [-x,  y,  z], [-x,  y, -z], [-x, -y,  z], [-x, -y, -z],
        ])

    elif isinstance(shape, coal.Cylinder):
        r = shape.radius
        h = shape.halfLength
        theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
        circle = np.column_stack([r * np.cos(theta), r * np.sin(theta)])
        top = np.column_stack([circle, np.full(n, h)])
        bot = np.column_stack([circle, np.full(n, -h)])
        verts = np.vstack([top, bot])

    elif isinstance(shape, coal.Cone):
        r = shape.radius
        h = shape.halfLength
        theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
        base = np.column_stack([r * np.cos(theta), r * np.sin(theta),
                                np.full(n, -h)])
        apex = np.array([[0.0, 0.0, h]])
        verts = np.vstack([base, apex])

    elif isinstance(shape, coal.Sphere):
        verts = _sphere_points(shape.radius, n)

    elif isinstance(shape, coal.Ellipsoid):
        rx, ry, rz = shape.radii
        unit = _sphere_points(1.0, n)
        verts = unit * np.array([rx, ry, rz])

    elif isinstance(shape, coal.Capsule):
        r = shape.radius
        h = shape.halfLength
        # hemisphere + cylinder ring
        unit = _sphere_points(1.0, n)
        top_hemi = unit[unit[:, 2] >= 0] * r + [0, 0, h]
        bot_hemi = unit[unit[:, 2] <= 0] * r + [0, 0, -h]
        theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
        ring = np.column_stack([r * np.cos(theta), r * np.sin(theta),
                                np.zeros(n)])
        verts = np.vstack([top_hemi, bot_hemi,
                           ring + [0, 0, h], ring + [0, 0, -h]])

    else:
        raise TypeError(f"Unsupported shape type: {type(shape)}")

    return _points_to_convex(verts)


def _sphere_points(radius, n):
    """Fibonacci sphere sampling."""
    indices = np.arange(n * n // 2)  # enough points
    # Use a simpler grid approach
    u = np.linspace(0, 2 * np.pi, n, endpoint=False)
    v = np.linspace(0, np.pi, n // 2 + 1)
    theta, phi = np.meshgrid(u, v)
    theta, phi = theta.ravel(), phi.ravel()
    x = radius * np.sin(phi) * np.cos(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(phi)
    return np.column_stack([x, y, z])


def _points_to_convex(points):
    """Build a BVHModel from a point cloud and compute its convex hull."""
    bvh = coal.BVHModelOBBRSS()
    bvh.beginModel(0, len(points))
    bvh.addVertices(points)
    bvh.endModel()
    bvh.buildConvexHull(True, "Qt")
    return bvh.convex

Apply function to convert all collision geometries to convex meshes.

In [78]:
# Apply function to all geometry objects in collision_model
for geom_obj in robot.collision_model.geometryObjects:
    geom = geom_obj.geometry
    if isinstance(geom, coal.ShapeBase):
        convex = primitive_to_convex(geom)
        geom_obj.geometry = convex
    elif isinstance(geom, coal.BVHModelBase):
        geom.buildConvexHull(True, 'Qt')
        geom_obj.geometry = geom.convex

# Update robot data
robot.data = robot.model.createData()
robot.collision_data = robot.collision_model.createData()
robot.visual_data = robot.visual_model.createData()

## Display robot (showing collision geometry instead of visual geometry)

Display robot (and obstacles) in browser.

In [79]:
# Create a visualizer
vis = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
robot.setVisualizer(vis, init=False)
vis.initViewer(open=True)
vis.loadViewerModel()

# Pause to allow meshcat initialization to finish
time.sleep(0.1)

# Choose what to display
vis.displayFrames(False)
vis.displayVisuals(True)
vis.displayCollisions(False)

# Pause to allow meshcat initialization to finish
time.sleep(0.1)

# Change the camera view
vis.setCameraPosition(np.array([0., 2., 0.75]))
vis.setCameraTarget(np.array([0., 0., 0.75]))

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


Put the robot at its "neutral" configuration.

In [80]:
# Get and show the neutral configuration (most likely all zeros)
q = pin.neutral(robot.model)
print(f'{q = }')

# Forward kinematics
# - Compute the placement of all joint frames (modifies robot.data but not robot.model or q)
pin.forwardKinematics(robot.model, robot.data, q)
# - Compute the placement of all link frames (modifies robot.data but not robot.model)
pin.updateFramePlacements(robot.model, robot.data)


q = array([0., 0., 0., 0., 0., 0.])


In [81]:
# Callback function that receives all slider values
def on_slider_change(change):
    global q
    q = np.deg2rad(np.array([slider.value for slider in sliders]))
    pin.forwardKinematics(robot.model, robot.data, q)
    pin.updateFramePlacements(robot.model, robot.data)
    vis.display(q)

# Create sliders for each joint
sliders = []
for i in range(robot.nq):
    slider = widgets.FloatSlider(
        value=0,
        min=-180,
        max=180,
        step=0.1,
        description=f'Joint {i+1}:',
        continuous_update=True,  # Only update on release
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
        layout=widgets.Layout(width='600px')  # Set slider width
    )
    # Attach the same callback to each slider
    slider.observe(on_slider_change, names='value')
    sliders.append(slider)

# Display all sliders
display(widgets.VBox(sliders))

# Initialize robot
on_slider_change(None)

Switch from showing "visual" geometry to showing "collision" geometry.

In [82]:
vis.displayVisuals(False)
vis.displayCollisions(True)

## Define functions to check for collision

Define a function that returns true if there is a collision and false otherwise.

In [83]:
def boolean_collision(q):
    pin.updateGeometryPlacements(
        robot.model,
        robot.data,
        robot.collision_model,
        robot.collision_data,
        q)
    return pin.computeCollisions(robot.collision_model, robot.collision_data, False)

Define a function that returns the (minimum) distance to collision in the workspace. (Note that `computeDistances` will also compute the penetration distance in case there is collision - we ignore this result for now.)

In [84]:
def distance_collision(q):
    if boolean_collision(q):
        return 0.
    else:
        # Compute the distance between each collision pair and
        # get the index of the pair with the smallest distance
        i = pin.computeDistances(robot.collision_model, robot.collision_data)
        # Return that smallest distance
        return robot.collision_data.distanceResults[i].min_distance

## Use our own implementation of RRT-Connect to plan a path

Define a simple tree data structure.

Here is how to create a root node with some configuration `q_start`:
```python
tree = Node(q_start)
```

Here is how to create a child of this root node with some other configuration `q1`:
```python
n1 = Node(q1, parent=tree)
```

Here is how to iterate through the entire tree:
```python
print('ALL NODES IN TREE')
for n in tree:
    print(f' q = {q}')
```

In [85]:
class Node:
    def __init__(self, q, parent=None):
        self.q = q
        self.parent = parent
        self.children = []
        if parent is not None:
            parent.children.append(self)
    
    def __iter__(self):
        yield self
        for child in self.children:
            yield from child

Implement RRT-Connect.

In [86]:
def find_closest_node_in_tree(tree, q):
    """
    Returns the node in the tree whose configuration is closest to q.
    """
    closest_node = None
    dist_min = float('inf')
    for node in tree:
        d = np.linalg.norm(node.q - q)
        if d < dist_min:
            dist_min = d
            closest_node = node
    return closest_node

def boolean_collision_edge(q1, q2, res=1e-3):
    """
    Returns True if the straight-line path from q1 to q2 has a collision
    (checked at resolution 1e-3) and False otherwise.
    """
    dist = np.linalg.norm(q1-q2)
    T = round(dist/res)
    for t in range(T):
        q = q1 + (q2 - q1) / dist * res * t
        if boolean_collision(q):
            return True
    return False

def extend(tree, q, epsilon = 0.1):
    node_closest = find_closest_node_in_tree(tree, q)
    v = q - node_closest.q
    v_norm = np.linalg.norm(v)
    if v_norm <= epsilon:
        q_new = q
        scenario = 1 # reached
    else:
        q_new = node_closest.q + v / v_norm * epsilon
        scenario = 2 # advanced
    if not boolean_collision_edge(q_new, node_closest.q):
        node_new = Node(q_new, parent = node_closest) # the tree is updated
    else:
        node_new = None
        scenario = 3 # trapped (collision)
    return scenario, node_new

def connect(tree, q):
    while True:
        scenario, node_new = extend(tree, q)
        if scenario == 1 or scenario == 3:
            break
    return scenario, node_new # can only be 'reached' or 'trapped'
    
def plan_path(q_start, q_goal, max_iters=1000):
    """
    Returns a path - a 2d numpy array with a waypoint in each row - from
    q_start to q_goal, or None if no path is found before the maximum number
    of iterations has been reached.
    """

    # Create trees rooted at start and goal
    tree_a = Node(q_start)
    tree_b = Node(q_goal)

    # Iterate for at most max_iters iterations
    for iter in range(max_iters):
        # Sample a random configuration that satisfies joint limits, keeping
        # it only if not in collision
        while True:
            q_rand = pin.randomConfiguration(robot.model)
            if not boolean_collision(q_rand):
                break

        scenario_a, node_new_a = extend(tree_a, q_rand)
        if scenario_a != 3 and node_new_a is not None:
            scenario_b, node_new_b = connect(tree_b, node_new_a.q)
            if scenario_b == 1:
                # If the configuration was connected to both forward and backward trees,
                # then a solution has been found - return it as a list of waypoints                
                path_a = []
                node = node_new_a
                while True:
                    path_a.append(node.q)
                    if node.parent is None:
                        break
                    node = node.parent
                
                path_b = []
                node = node_new_b
                while True:
                    path_b.append(node.q)
                    if node.parent is None:
                        break
                    node = node.parent
                
                if np.all(path_a[-1] == q_start):
                    path_a = path_a[::-1]
                    path = path_a + path_b
                elif np.all(path_b[-1] == q_start):
                    path_b = path_b[::-1]
                    path = path_b + path_a
                return path

        # Swap tree_a and tree_b     
        tree_a, tree_b = tree_b, tree_a
    
    # Declare failure if no path was found
    print('FAILURE (after {max_iters} iterations)')
    return None

Apply implementation of RRT-Connect to plan a path.

In [87]:
# Define start and goal configurations
q_start = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
q_goal = np.array([np.pi/2, -np.pi/2, 0.0, 0.0, 0.0, 0.0])

# Plan path
path = plan_path(q_start, q_goal, max_iters=1000)

print(path)

[array([0., 0., 0., 0., 0., 0.]), array([ 0.01692321, -0.0607631 , -0.00645459,  0.05723926,  0.00409496,
       -0.05183327]), array([ 0.09144806, -0.12722866, -0.0057514 ,  0.05409638,  0.00753857,
       -0.04935413]), array([ 0.1659729 , -0.19369422, -0.00504822,  0.05095351,  0.01098218,
       -0.04687499]), array([ 0.24049775, -0.26015978, -0.00434503,  0.04781063,  0.0144258 ,
       -0.04439585]), array([ 0.31502259, -0.32662534, -0.00364184,  0.04466775,  0.01786941,
       -0.04191672]), array([ 0.38954744, -0.3930909 , -0.00293865,  0.04152488,  0.02131302,
       -0.03943758]), array([ 0.43787883, -0.38992982, -0.0160308 ,  0.09150615,  0.05827177,
        0.02071726]), array([ 0.51827817, -0.44345327, -0.00347618,  0.08767662,  0.05383376,
       -0.00117351]), array([ 0.60235813, -0.45853767,  0.00536722,  0.04508557,  0.03478068,
       -0.02233476]), array([ 0.68643809, -0.47362207,  0.01421062,  0.00249452,  0.01572761,
       -0.04349601]), array([ 0.70310865, -0.476

Define a function to interpolate the path.

In [90]:
def interpolate_path(path, res=1e-2):
    """
    Adds intermediate waypoints so that the distance between each consecutive
    waypoint is at most res. Should return a *copy* of the path with the result
    - should not modify the path in place.
    """
    path_interp = [path[0]]
    
    for i in range(len(path) - 1):
        q1 = path[i]
        q2 = path[i + 1]
        dist = np.linalg.norm(q2 - q1)
        if dist > res:
            for t in range(int(np.ceil(dist / res))):
                q_interp = q1 + t * (q2 - q1) / dist * res
                path_interp.append(q_interp)
    
    return path_interp

Define a function to smooth the path.

In [ ]:
def get_path_lengths(path):
    """
    Returns a 1d array with the same length as path and with the
    cumulative sum of path lengths in each entry.

    Suppose path has waypoints q1, ..., qn. Then, the array that
    is returned should be as follows:

    [
        0.,
        distance from q1 to q2,
        (distance from q1 to q2) + (distance from q2 to q3),
        ...
    ]
    """
    path_lengths = [0]
    dist_cumsum = 0
    for i in range(len(path)-1):
        q1 = path[i]
        q2 = path[i + 1]
        dist_cumsum += np.linalg.norm(q2 - q1)
        path_lengths.append(dist_cumsum)

    return path_lengths

def smooth_path(path, max_iters=100):
    """
    Applies shortcutting max_iters times to smooth the path. Should
    return a *copy* of the path with the result - should not modify
    the path in place.
    """
    # Compute path lengths
    d = get_path_lengths(path)

    smooth_path = path.copy()

    # Iterate max_iters times
    for i in range(max_iters):
        # Sample two points q1 and q2 along the path, keeping them only
        # if they are not on the same edge
        while True:
            ind = random.sample(range(len(smooth_path)), 2)
            if abs(ind[0] - ind[1]) > 1:
                q1 = smooth_path[ind[0]]
                q2 = smooth_path[ind[1]]
                break
        
        # Continue only if the straight-line path between q1 and q2 is collision-free
        if not boolean_collision_edge(q1, q2):
            # Replace all waypoints between q1 and q2 with the new straightline path
            ind1 = min(ind[0], ind[1])
            ind2 = max(ind[0], ind[1])
            smooth_path = smooth_path[0:ind1+1] + smooth_path[ind2:]

        # Recompute path lengths
        d = get_path_lengths(smooth_path)
    
    return smooth_path

Visualize the original path after interpolation.

In [93]:
# Interpolate original path
path_original = interpolate_path(path)

# Visualize the result (checking for collision to be sure)
for q in path_original:
    vis.display(q)
    if boolean_collision(q):
        raise Exception('collision!')
    time.sleep(0.025)

Visualize the smoothed path after interpolation.

In [98]:
# Smooth original path
path_smoothed = smooth_path(path)

# Interpolate smoothed path
path_smoothed = interpolate_path(path_smoothed)

# Visualize the result (checking for collision to be sure)
for q in path_smoothed:
    vis.display(q)
    if boolean_collision(q):
        raise Exception('collision!')
    time.sleep(0.025)

IndexError: list index out of range